In [18]:
import json
from nltk.corpus import wordnet as wn
from collections import defaultdict
import editdistance

# --- Configuration ---
# 1.0 = Perfect Interchangeability (Rare)
# 0.5 = Interchangeable in half of their contexts
THRESHOLD = 1 

def extract_vocab(json_path):
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except FileNotFoundError:
        return []
        
    vocab_keys = data.get('model', {}).get('vocab', {}).keys()
    
    # Fast set comprehension for filtering
    return {w for w in vocab_keys 
            if len(w) > 3 and w.isalpha() and not w.startswith("##")}

def find_contextual_synonyms(vocab_set):
    print(f"Building indices for {len(vocab_set)} words...")
    
    # 1. Build Lookup Tables (The "Inverted Index")
    # specific_word -> set of Synset IDs (e.g., 'dog' -> {'dog.n.01', 'frump.n.01'...})
    word_to_synsets = {}
    
    # synset_id -> list of words that belong to it
    synset_to_words = defaultdict(list)
    
    for word in vocab_set:
        # Get all Synset IDs (meanings) for this word
        # We use IDs (syn.name()) because they are unique strings
        synsets = set(syn.name() for syn in wn.synsets(word))
        
        if synsets:
            word_to_synsets[word] = synsets
            for sid in synsets:
                synset_to_words[sid].append(word)
                
    print("Calculating overlaps...")
    
    results = []
    processed_pairs = set()

    # 2. Iterate through words and find their "Contextual Twins"
    for word_a, contexts_a in word_to_synsets.items():
        
        # OPTIMIZATION: Only look at words that share at least ONE context.
        # This avoids comparing "apple" vs "zebra".
        candidates = set()
        for sid in contexts_a:
            candidates.update(synset_to_words[sid])
            
        # Remove self from candidates
        candidates.discard(word_a)
        
        for word_b in candidates:
            # Sort pair to avoid duplicates: (A, B) and (B, A)
            pair_key = tuple(sorted((word_a, word_b)))
            if pair_key in processed_pairs:
                continue
            
            processed_pairs.add(pair_key)
            
            # 3. Calculate Similarity (Jaccard Index)
            contexts_b = word_to_synsets[word_b]
            
            intersection = len(contexts_a.intersection(contexts_b))
            union = len(contexts_a.union(contexts_b))
            
            score = intersection / union
            edit_d_ratio = editdistance.eval(word_a, word_b) / max(len(word_a), len(word_b))
            
            if score >= THRESHOLD and edit_d_ratio > 0:
                results.append({
                    "word_a": word_a,
                    "word_b": word_b,
                    "score": round(score, 2),
                    "shared_contexts": intersection,
                    "total_contexts": union
                })

    return results

# --- Execution ---

# 1. Load Vocab
vocab = extract_vocab("./bert_tokenizer_uncased/tokenizer.json")

# (Dummy data for testing)
if not vocab:
    vocab = {"start", "begin", "commence", "car", "automobile", "speech", "talk", "run", "sprint"}

# 2. Find Pairs
interchangeable_pairs = find_contextual_synonyms(vocab)

# 3. Sort by Score (Most interchangeable first)
interchangeable_pairs.sort(key=lambda x: x['score'], reverse=True)

print(f"\nFound {len(interchangeable_pairs)} highly interchangeable pairs.\n")
print(f"{'Word A':<15} | {'Word B':<15} | {'Score':<5} | {'Shared Meanings'}")
print("-" * 60)

for p in interchangeable_pairs[:20]:
    print(f"{p['word_a']:<15} | {p['word_b']:<15} | {p['score']:<5} | {p['shared_contexts']}/{p['total_contexts']}")

Building indices for 17582 words...
Calculating overlaps...

Found 2437 highly interchangeable pairs.

Word A          | Word B          | Score | Shared Meanings
------------------------------------------------------------
kisses          | kiss            | 1.0   | 6/6
examining       | examine         | 1.0   | 5/5
examining       | examined        | 1.0   | 5/5
generating      | generate        | 1.0   | 4/4
generating      | generated       | 1.0   | 4/4
advocate        | advocates       | 1.0   | 4/4
workers         | worker          | 1.0   | 4/4
prisoners       | prisoner        | 1.0   | 1/1
adviser         | advisor         | 1.0   | 1/1
adviser         | consultant      | 1.0   | 1/1
adviser         | advisers        | 1.0   | 1/1
adviser         | advisors        | 1.0   | 1/1
lobbied         | lobbying        | 1.0   | 1/1
spear           | spears          | 1.0   | 4/4
pump            | pumps           | 1.0   | 11/11
defended        | defend          | 1.0   | 7/7
slaves

In [19]:
unique_words = dict()
unique_words_related = {}
for pair in interchangeable_pairs:
    unique_words[pair['word_a']] = 0
    unique_words[pair['word_b']] = 0
    unique_words_related[pair['word_a']] = set()
    unique_words_related[pair['word_b']] = set()
print(f"Number of unique words in filtered pairs: {len(unique_words)}")

for pair in interchangeable_pairs:
    unique_words[pair['word_a']] += 1
    unique_words[pair['word_b']] += 1
    unique_words_related[pair['word_a']].add(pair['word_b'])
    unique_words_related[pair['word_b']].add(pair['word_a'])


sorted_unique_words = sorted(unique_words.items(), key=lambda x: x[1], reverse=True)

Number of unique words in filtered pairs: 4177


In [20]:
# print the words withthe highest counts

print(f"{'Word':<20} | {'Count':<10} | {'Related Words'}")
print("-" * 60)
for word, count in sorted_unique_words:
    related_words = ", ".join(sorted(unique_words_related[word]))
    print(f"{word:<20} | {count:<10} | {related_words}")

Word                 | Count      | Related Words
------------------------------------------------------------
realizes             | 5          | realise, realises, realising, realize, realizing
realises             | 5          | realise, realising, realize, realizes, realizing
realizing            | 5          | realise, realises, realising, realize, realizes
realize              | 5          | realise, realises, realising, realizes, realizing
realising            | 5          | realise, realises, realize, realizes, realizing
realise              | 5          | realises, realising, realize, realizes, realizing
adviser              | 4          | advisers, advisor, advisors, consultant
advisor              | 4          | adviser, advisers, advisors, consultant
consultant           | 4          | adviser, advisers, advisor, advisors
advisers             | 4          | adviser, advisor, advisors, consultant
advisors             | 4          | adviser, advisers, advisor, consultant
chal

In [21]:
#iterate through the most common words and remove all related words

mapping = dict()

removed_words = set()
while len(sorted_unique_words) > 0:
    
    word, count = sorted_unique_words[0]
    if count == 0:
        break
    if word not in unique_words:
        continue

    #Pop the top word from the list
    unique_words.pop(word, None)
    related_words = set(unique_words_related[word])

    # Remove related words from all other entries
    for related_word in related_words:
        mapping[related_word] = word
        removed_words.add(related_word)
        #remove the keys for the related words
        unique_words.pop(related_word, None)
        unique_words_related.pop(related_word, None)
        #remove the instance of the related word from all other related word sets
        for word2 in unique_words:
            if related_word in unique_words_related[word2]:
                unique_words_related[word2].remove(related_word)
                unique_words[word2] -= 1
    sorted_unique_words = sorted(unique_words.items(), key=lambda x: x[1], reverse=True)
    

print(f"Number of removed words: {len(removed_words)}")
print(f"Removed words: {', '.join(sorted(removed_words))}")

Number of removed words: 2186
Removed words: abilities, abraham, abuse, accepts, accident, accounts, achieve, achievement, achieving, acknowledge, action, activities, actor, actress, adapt, adaptations, added, addition, address, adherents, administer, administrators, admissions, admits, admitted, admitting, adopt, advantage, adventures, advertisement, advisers, advisor, advisors, advocated, advocates, aeronaut, affect, afforded, afterwards, agencies, agent, agree, agreements, agrees, aiming, airfields, airlines, airports, album, alliances, allowed, allowing, allows, alloy, altitude, aluminum, amassed, ambassador, ambitions, amendment, americas, amir, amount, analysts, ancestor, angels, angles, animation, animator, announces, announcing, anonym, answers, antibody, apartments, appeal, appear, appearance, appears, applaud, applications, applies, applying, appoint, appointments, apprentice, approaches, arab, archaeologist, architect, area, argue, argues, arguments, arise, arising, armour, 

In [22]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./bert_tokenizer_uncased")
model_state = tokenizer.get_vocab()
for word in removed_words:
    model_state.pop(word, None)

print(model_state)
vocab_list = [token for token, idx in sorted(model_state.items(), key=lambda x: x[1])]
print(vocab_list)
print(len(vocab_list), "words in new vocab")

import json
with open("filtered_tokenizer_vocab_wordnet.json", "w", encoding="utf-8") as f:
    json.dump(vocab_list, f, ensure_ascii=False, indent=2)


with open("removed_words_mapping_wordnet.json", "w", encoding="utf-8") as f:
    json.dump(mapping, f, ensure_ascii=False, indent=2)

{'##kary': 24057, '沐': 2704, '##ici': 9376, 'preference': 18722, 'caliphate': 28833, '##nam': 8746, '##yne': 25029, '尋': 2084, '##п': 4191, 'manz': 23330, 'exchanged': 19729, 'bute': 29826, 'exagger': 18273, 'cancell': 24926, '總': 3203, 'smallpox': 23313, '[MASK]': 4, 'liam': 27536, 'ժ': 340, 'room': 7190, 'another': 6033, 'dich': 29933, 'quasi': 27033, 'shower': 23421, '##fold': 18259, 'ind': 5496, 'taped': 26211, 'logging': 25922, 'fuj': 24245, 'affairs': 10569, 'subordinates': 29155, 'impending': 22742, 'utah': 14505, '##የ': 4940, '980': 27196, '1782': 27228, 'bordered': 25920, '168': 13200, 'vs': 12498, 'heinz': 29940, 'hammersmith': 27049, 'austen': 18088, '##ilus': 19074, 'torture': 18718, '##ware': 10062, 'foremost': 24206, 'consumer': 17108, 'ransom': 27839, 'greeted': 21008, 'ಯ': 673, 'sable': 29134, 'mete': 11400, '##ゅ': 4838, 'williamson': 23423, 'gul': 19127, '誘': 3523, 'া': 558, 'spotted': 12776, 'rollins': 25309, '##ylvania': 9231, 'senate': 9205, 'incarn': 19585, 'vermon